In [ ]:
import requests

# API REQUEST

In [ ]:
# request
def make_request(endpoint, payload= None):
  return requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',headers={'token': 'anPgrgSWrWgUYFXULhgBXbwIMcbuJWps'},params=payload)

In [ ]:
response = make_request('datasets', {'startdate':'2018-10-01'})
response.status_code # 200 means Okay

# Getting Keys

In [ ]:
response.json().keys() # getting Keys

In [ ]:
response.json()["metadata"]

# What data in the results

In [ ]:
response.json()['results'][0].keys()

# Parsing

In [ ]:
[(data['id'], data['name']) for data in response.json()["results"] ]

# Categories

In [ ]:
response = make_request('datacategories', payload={'datasetid' : 'GHCND'})
response.status_code

In [ ]:
response.json()["results"]

# Grabing Data type ID // Temperature Category

In [ ]:
response = make_request('datatypes', payload={'datacategoryid' : 'TEMP', 'limit' : 100})
response.status_code

In [ ]:
[(datatype['id'], datatype['name']) for datatype in response.json()['results']][-5:]

# Determine which Location of the category

In [ ]:
response = make_request('locationcategories', payload={"datasetid":"GHCND"})
response.status_code

In [ ]:
import pprint # for more readability of the json()
pprint.pprint(response.json())

# NYC Location ID

In [ ]:
def get_item(name,  what, endpoint, start=1, end=None):
  mid = (start + (end if end else 1)) // 2
  name = name.lower()
  payload = { 'datasetid': 'GHCND',
             'sortfield': 'name',
              'offset': mid,
              'limit':1}

  response = make_request(endpoint, {**payload, **what})

  if response.ok:
    end = end if end else response.json()["metadata"]['resultset']['count']

    current_name = response.json()["results"][0]['name'].lower()

    if name in current_name:
      return response.json()['results'][0]

    else:
      if start >= end:
        return {}
      elif name < current_name:
        return get_item(name, what, endpoint, start, mid -1)
      elif name > current_name:
        return get_item(name, what, endpoint, mid + 1, end)

  else:
    print(f'Response not OK, status: {response.status_code}')

def get_location(name):
  return get_item(name, {'locationcategoryid': 'CITY'}, 'locations')

In [ ]:
nyc = get_location('New York')
nyc

# Station ID for Central Park

In [ ]:
central_park = get_item('NY City Central Park', {'locationid' : nyc['id']}, 'stations')
central_park

# Temperature Data Request

In [ ]:
# get NYC daily summaries data
response = make_request('data',{
        'datasetid' : 'GHCND',
        'stationid' : central_park['id'],
        'locationid' : nyc['id'],
        'startdate' : '2018-10-01',
        'enddate' : '2018-10-31',
        'datatypeid' : ['TMIN', 'TMAX', 'TOBS'], # temperature at time of observation, min, and max
        'units' : 'metric',
        'limit' : 1000
    }
)
response.status_code

## DATA FRAMES

In [ ]:
import pandas as pd
df = pd.DataFrame(response.json()['results'])
df.head()

In [ ]:
df.datatype.unique()

In [ ]:
if get_item('NY City Central Park', {'locationid':nyc['id'], 'datatypeid': 'TOBS'}, 'stations'):
  print("Found!")

# Different Location

In [ ]:
laguardia = get_item(
'LaGuardia', {'locationid' : nyc['id']}, 'stations'
)
laguardia

In [ ]:
response = make_request(
'data',
{
'datasetid' : 'GHCND',
'stationid' : laguardia['id'],
'locationid' : nyc['id'],
'startdate' : '2018-10-01',
'enddate' : '2018-10-31',
'datatypeid' : ['TMIN', 'TMAX', 'TAVG'], # temperature at time of observation, min, and max
'units' : 'metric',
'limit' : 1000
}
)
response.status_code

In [ ]:
df = pd.DataFrame(response.json()['results'])
df.head()

In [ ]:
df.datatype.value_counts()

In [ ]:
df.to_csv('/content/nyc_temperatures.csv', index=False)

#**END**